In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessio

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier , GradientBoostingClassifier , ExtraTreesClassifier
from sklearn.svm import SVC , LinearSVC
from sklearn.model_selection import KFold, train_test_split

In [6]:
#read data
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
data = [train , test]

In [7]:
train.shape , test.shape

In [8]:
train.head()

In [9]:
#describe categorical features
train.describe(include = ["O"])

all names are unique
there are two unique Sex values (male , female) , male is most frequently in train dataset(65%)
there is dublicates in ticket feature
cabin has dublicate (more than one passenger share same cabin)
"s" is the most frequent distination

In [10]:
train.info()

age , cabin and embarked have null values
catgorical : name , sex , Ticket , cabin , Embarked , Pclass
numerical : age , fare , parch , sibsp

In [12]:
#add new features
for d in data : 
    d["NameLen"] = d["Name"].apply(lambda x : len(x))
    
train.head()

In [13]:
# add family size
for d in data:
    d["FamSize"] = d["SibSp"] + d["Parch"] + 1

train.head()

In [15]:
# is alone or not
for d in data : 
    d["IsAlone"] = d["FamSize"].apply(lambda x : 0 if x != 1 else 1)

train.head()

In [16]:
# fill embarked nulls with "S"
for d in data:
    d["Embarked"].fillna("S" , inplace = True)

train["Embarked"].isnull().sum() , test["Embarked"].isnull().sum()

In [17]:
# remove cabin features
train = train.drop("Cabin" , axis = 1)
test = test.drop("Cabin" , axis  = 1)
data = [train , test]
train.head()

In [18]:
g = sns.FacetGrid(train , col = "Survived" , row = "Pclass")
g.map(plt.hist , "Age")

In [19]:
g = sns.FacetGrid(train , col = "Sex" , row = "Pclass")
g.map(plt.hist , "Age")

In [20]:
g = sns.FacetGrid(train , col = "Embarked")
g.map(plt.hist , "Age")

In [21]:
g = sns.FacetGrid(train , col = "Pclass")
g.map(plt.hist , "Age")

In [23]:
# fill age nulls

ages = np.zeros((2,3))



for d in data:
    for i in range(0 , 2):
        for j in range(0 , 3): 
            a = d[(d["Pclass"] ==j+1) & (d["Sex"] == i)]["Age"].dropna()
            ages[i,j] = a.median()
            
    for i in range(0 , 2):
        for j in range(0,3):
            d.loc[(d["Age"].isnull())&(d["Pclass"] == j+1)&(d["Sex"] == i)] = ages[i,j]
            

train.info()

In [24]:
data[0]

In [25]:
train.info()

In [26]:
train[["Sex" , "Survived"]].groupby("Sex" , as_index = False).mean()

female has 74% chance to survive
male has 18% chance to survive

In [27]:
train[["Embarked" , "Survived"]].groupby("Embarked" , as_index = False).mean().sort_values(by = "Survived")

In [28]:
sns.countplot(data  = train ,x = "Embarked")

In [29]:
# replaec name with numbers
for d in data:
    d["Sex"] = d["Sex"].replace({"male" : 0 , "female" : 1})
    d["Sex"] = d["Sex"].astype(int)
    
train.head()

In [30]:
#convert embarked 
for d in data:
    d["Embarked"] = d["Embarked"].replace({"S" : 0 , "C" : 1 , "Q" : 2})
    d["Embarked"] = d["Embarked"].astype(int)
    
train.head()

In [31]:
sns.distplot(train["Age"])

In [33]:
# remove missing values

for d in data:
    age_mean = d["Age"].mean()
    age_std = d["Age"].std()
    n_rnd = d["Age"].isnull().sum()
    rnd_values = np.random.randint(age_mean-age_std , age_mean+age_std , n_rnd)
    d["Age"][np.isnan(d["Age"])] = rnd_values
    d["Age"] = d["Age"].astype(int)

train.head()

In [34]:
train.info()

In [35]:
test.info()

In [36]:
for d in data:
    d["Fare"] = d["Fare"].fillna(d["Fare"].mean())
test.info()

In [37]:
train["Survived"] = train.Survived.astype(int)
g = sns.FacetGrid(train , col = "Survived" , size = 2.2 , aspect = 1.2)
g.map(plt.hist , "NameLen" , bins = 20)

In [38]:
train[["NameLen" , "Survived"]].groupby("NameLen" , as_index = False).mean().sort_values(
by = "Survived" , ascending = False)

In [39]:
plt.scatter(x = train["NameLen"] , y = train["Survived"])

there is no relation between NameLen and Survived so we will delete NameLen

In [40]:
train = train.drop("NameLen" , axis = 1)
test = test.drop("NameLen" , axis = 1)

data = [train , test]

train.head()

In [41]:
#drop PassengerId
train = train.drop("PassengerId" , axis = 1)
test = test.drop("PassengerId" , axis = 1)

data = [train , test]

train.head()

In [42]:
# separate title from name
for d in data:
    d["Title"] = d["Name"].str.extract("([a-zA-Z]+)\.")

train.head()

In [43]:
#delete name feature
train = train.drop("Name" , axis = 1)
test = test.drop("Name" , axis = 1)
data = [train , test]
train.head()

In [46]:
train[["Title" , "Survived"]].groupby("Title" , as_index = False).count().sort_values(
by = "Survived" , ascending = False)

In [47]:
for d in data:
    d["Title"] = d["Title"].replace(["Dr" , "Rev" , "Major","Col" , "Mlle","Mme","Ms","Capt","Lady","Jonkheer",
                                     "Dona" ,"Don","Countess","Sir"] , "Rare")
    d["Title"] = d["Title"].replace({"Master" : "Mr" , "Miss" : "Mrs"})
    
train.head()

In [48]:
train[["Title" , "Survived"]].groupby("Title" , as_index = False).mean().sort_values(
by = "Survived" , ascending = False)

In [49]:
for d in data :
    d["Title"] = d["Title"].replace({"Mrs" : 0 , "Mr" : 1 , "Rare":2})
    d["Title"] = d["Title"].astype(int)

train.head()

In [50]:
train = train.drop("Ticket" , axis = 1)
test = test.drop("Ticket" , axis = 1)

data = [train , test]
train.head()

In [51]:
#categoriza age
train["CatAge"] = pd.cut(train["Age"] , 5)
train.head()

In [52]:
train[["CatAge" , "Survived"]].groupby("CatAge").mean()

In [53]:
for d in data:
    d.loc[(d["Age"] <=16.0) , "Age"] = 0
    d.loc[(d["Age"] > 16)&(d["Age"]<=32) , "Age"] =1
    d.loc[(d["Age"] > 32)&(d["Age"]<=48) , "Age"] =2
    d.loc[(d["Age"] > 48)&(d["Age"]<=64) , "Age"] =3
    d.loc[(d["Age"] > 64), "Age"] =4
    
    d["Age"] = d["Age"].astype(int)
    
train.head()

In [54]:
train = train.drop("CatAge" , axis = 1)
data = [train , test]
train.head()

In [56]:
for d in data:
    d["SibSp"] = d["SibSp"].astype(int)
    d["Parch"] = d["Parch"].astype(int)
    d["Pclass"] = d["Pclass"].astype(int)
    d["FamSize"] = d["FamSize"].astype(int)
    d["IsAlone"] = d["IsAlone"].astype(int)
    
test.head()

In [57]:
g = sns.FacetGrid(train , col = "Survived" )
g.map(plt.hist , "Fare",bins = 10)

In [58]:
plt.scatter(train["Survived"] , train["Fare"])

In [59]:
train["catFare"] = pd.qcut(train["Fare"] , 4)
train[["catFare" , "Survived"]].groupby("catFare" , as_index = False).mean()

In [60]:
for d in data : 
    d.loc[(d["Fare"] <=7.91) , "Fare"] = 0
    d.loc[(d["Fare"] >7.91) & (d["Fare"] <= 14.454) , "Fare"] = 1
    
    d.loc[(d["Fare"] >14.454) & (d["Fare"] <= 31.0) , "Fare"] = 2
    
    d.loc[(d["Fare"] > 31.0) , "Fare"] = 3
    
    d["Fare"] = d["Fare"].astype(int)
    

train["Fare"].unique()

In [61]:
test["Fare"].unique()

In [62]:
train = train.drop("catFare" , axis = 1)
train.head()

In [63]:
test.head()

In [64]:
train.info()

In [65]:
test.info()

In [66]:
plt.figure(figsize = (14 , 12))
sns.heatmap(train.corr() , linewidth = 0.1, vmax = 1 , square= True , cmap= plt.cm.RdBu , linecolor = "white" , annot = True)

insights

high correlation between pclass and fare
high corralation between sex and title
high correlation between sibsp, famsize and isalone
high correlation between parch, famsize and isalone

decisions:

drop pclass
drop title
drop sibsp and famsize
drop parch

In [69]:
test.head()

In [72]:
from sklearn.model_selection import KFold

ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(n_splits = NFOLDS, random_state=SEED , shuffle = True).split(train)


class SklearnHelper(object):
    def __init__(self , clf , seed = 0 , params = None):
        params["random_state"] = seed
        self.clf = clf(**params)
    
    def train(self , x_train , y_train):
        return self.clf.fit(x_train , y_train)
    
    def predict(self , x):
        return self.clf.predict(x)
    
    def feature_importances(self , x , y):
        print(self.clf.fit(x,y).feature_importances_)
    

In [73]:
    
def get_oof(clf , x_train , y_train , x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i , (train_idx , test_idx) in enumerate(kf):
        
        x_tr = x_train[train_idx]
        y_tr = y_train[train_idx]
        x_te = x_train[test_idx]
        
        clf.train(x_tr , y_tr)
        
        oof_train[test_idx] = clf.predict(x_te)
        oof_test_skf[i , : ] = clf.predict(x_test)
    
    oof_test[:] = oof_test_skf.mean(axis = 0)
    return oof_train.reshape(-1,1),oof_test.reshape(-1 , 1)

In [76]:
from sklearn.ensemble import (RandomForestClassifier , AdaBoostClassifier,
                             GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC

In [78]:
# initial parametars of models

#random forest
rf_params = {"n_estimators" : 500,
     "max_depth" : 6,
     "min_samples_leaf" : 2,
     "n_jobs" : -1,
     "max_features" : "sqrt",
     "verbose" : 0}

#extra trees
et_params = {"n_jobs" : -1,
            "n_estimators" : 500,
            "max_depth" : 6,
            "min_samples_leaf" : 2,
            "verbose" : 0}

# adaboost
ada_params = {
    "n_estimators" : 500
}


# gradiant boosting
gb_params = {
    "n_estimators" : 500,
    "verbose" : 0,
    "max_depth" : 5,
    "min_samples_leaf" : 2
}



# support vector classifier

svc_params = {
    
    "kernel" : "linear",
    "C" :  0.025
}

In [80]:
rf = SklearnHelper(clf = RandomForestClassifier , seed = SEED,params = rf_params)
et = SklearnHelper(clf = ExtraTreesClassifier , seed = SEED , params = et_params)
ada = SklearnHelper(clf = AdaBoostClassifier , seed = SEED , params = ada_params)
gb = SklearnHelper(clf = GradientBoostingClassifier , seed = SEED ,params = gb_params)
svc = SklearnHelper(clf = SVC ,seed = SEED , params = svc_params )

In [82]:
y_train = train["Survived"].ravel()
x_train = train.drop("Survived" , axis = 1)
x_train = x_train.values
x_test = test.values

In [83]:
re_oof_train , re_oof_test = get_oof(rf , x_train , y_train , x_test)
et_oof_train , et_oof_test = get_oof(et , x_train , y_train , x_test)
ada_oof_train , ada_oof_test = get_oof(ada , x_train , y_train , x_test)
gb_oof_train , gb_oof_test = get_oof(gb , x_train , y_train , x_test)
svc_oof_train , svc_oof_test = get_oof(svc , x_train , y_train , x_test)

In [84]:
rf_feature = rf.feature_importances(x_train , y_train)
et_feature = et.feature_importances(x_train , y_train)
ada_feature = ada.feature_importances(x_train , y_train)
gb_feature = gb.feature_importances(x_train , y_train)

In [85]:
rf_feature = [0.62511991, 0.09523811, 0.16694568, 0.06370805, 0.04898826]
et_feature = [0.70711206, 0.05404866 ,0.13972851, 0.05030473, 0.04880604]
ada_feature = [0.056, 0.182 ,0.214, 0.114 ,0.074]
gb_feature = [0.67743602, 0.0858747 , 0.12030498, 0.06088641, 0.05549789]

In [86]:
cols = train.drop("Survived" , axis = 1).columns.values

feature_dataframe = pd.DataFrame({"features" : cols , 
                                 "random forest featueres importance" : rf_feature,
                                 "extra trees features imporatances" : et_feature,
                                 "adaboost_features importances" : ada_feature,
                                 "gradient boosting features importances" : gb_feature})

In [87]:
feature_dataframe

In [88]:
import plotly.graph_objs as go
import plotly.offline as py